# AutoGen Demo

**Microsoft AutoGen v0.4** - A framework for building conversational multi-agent AI applications.

## What Makes AutoGen Special?
AutoGen excels at **multi-agent orchestration** - enabling AI agents to have natural conversations and collaborate to solve complex problems.

## Required Environment Variables

```
OPENAI_API_KEY    # Your OpenAI API key (required)
```

Please refer to the [README](README.md) for instructions on setting up environment variables.

## This Demo Shows:
1. **Simple Query** - Basic agent interaction with OpenAI
2. **Tool Usage** - Function calling capabilities  
3. **Agent-to-Agent Communication** - Multiple agents collaborating (AutoGen's strength!)

In [ ]:
# Setup and imports
import os
import asyncio
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

# Import AutoGen components
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient

## Step 1: Load Model & Simple Query

In [ ]:
# Create OpenAI model client
# The API key is loaded from the OPENAI_API_KEY environment variable
model_client = OpenAIChatCompletionClient(
    model="gpt-5-mini",  # Cost-effective model
    api_key=os.getenv('OPENAI_API_KEY')  # Explicitly load from environment
)

# Create basic agent
food_agent = AssistantAgent(
    name="food_advisor",
    model_client=model_client,
    description="A helpful assistant for food recommendations."
)

In [ ]:
# Make a real API call through AutoGen
query = "What should we have for lunch in Singapore? Suggest some local food. Keep it concise. Just 3 bullets points max."

print(f"Query: {query}")

# Create the message and get response
async def get_food_recommendation():
    response = await food_agent.on_messages(
        [TextMessage(content=query, source="user")],
        cancellation_token=None
    )
    print(response.chat_message.content)

# Run the async function
await get_food_recommendation()

## Step 2: Simple Tool Definition & Usage

In [ ]:
# Define a simple calculator tool
async def calculate(expression: str) -> str:
    """Evaluate mathematical expressions.

    Args:
        expression: Math expression like "2 + 3 * 4"

    Returns:
        str: Calculation result
    """
    try:
        # Safety check for basic math operations only
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"

        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:
# Create agent with tool
math_agent = AssistantAgent(
    name="math_assistant",
    model_client=model_client,
    tools=[calculate],  # Add the calculator tool
    description="An assistant that can perform calculations."
)


In [ ]:
# Demonstrate real tool usage through AutoGen
math_query = "Calculate the total cost if lunch costs $12.50 per person for 8 people."

print(f"Query: {math_query}")
print("="*50)
print("\nAgent Response (with tool usage):")

async def calculate_lunch_cost():
    response = await math_agent.on_messages(
        [TextMessage(content=math_query, source="user")],
        cancellation_token=None
    )
    print(response.chat_message.content)

# Run the calculation
await calculate_lunch_cost()

## Step 3: Agent-to-Agent Communication (AutoGen's Strength!)

This is where AutoGen truly shines - orchestrating conversations between multiple specialized agents.

In [ ]:
# Create two specialized agents that will collaborate

# Research Agent - gathers information
research_agent = AssistantAgent(
    name="researcher",
    model_client=model_client,
    description="Specializes in gathering and providing information about food prices and options. Keep responses concise."
)

# Analysis Agent - processes and analyzes data
analysis_agent = AssistantAgent(
    name="analyst",
    model_client=model_client,
    tools=[calculate],
    description="Specializes in data analysis, calculations, and budget recommendations. Keep responses concise."
)


In [ ]:
# Demonstrate real agent-to-agent communication using RoundRobinGroupChat
print("🎯 Task: Plan a team lunch budget for our Singapore office (12 people)")

# Create a group chat for agent collaboration
async def multi_agent_collaboration():
    # Simple termination after 2 messages (one from each agent)
    termination = MaxMessageTermination(max_messages=2)

    # Create the group chat team
    team = RoundRobinGroupChat(
        participants=[research_agent, analysis_agent],
        termination_condition=termination
    )

    # Simpler, more focused task
    task = """Plan a team lunch budget for 12 people in Singapore.
    Research typical lunch prices (hawker centers vs restaurants) and calculate the total budget needed."""

    # Run the team collaboration
    result = await team.run(task=task)

    # Display the conversation
    for message in result.messages:
        print(f"{message.source.upper()}:")
        content = message.content
        print(f"  {content}\n")

# Run the multi-agent demo
await multi_agent_collaboration()

## Summary

### What We Demonstrated:

1. **✅ Simple Query** - Real API call to OpenAI through AutoGen
2. **✅ Tool Usage** - Agent automatically used calculator tool when needed
3. **✅ Agent Communication** - Multiple agents had a real conversation to solve a task

### AutoGen's Key Strength:
**Multi-agent orchestration** - AutoGen makes it incredibly easy for multiple AI agents to have natural conversations and work together, each bringing their specialized capabilities to solve problems collaboratively.

